In [ ]:
#importation des données dvf géolocalisées en csv

import os
import s3fs
import pandas as pd
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]

fs = s3fs.S3FileSystem(
    anon=True,
    client_kwargs={"endpoint_url": S3_ENDPOINT_URL}
)

with fs.open("renan/diffusion/dvf.csv", mode="rb") as f:
    df = pd.read_csv(f)

/tmp/ipykernel_43287/352770705.py:14: DtypeWarning: Columns (10,12,14,16,17,18,20,22,24,26,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


In [ ]:
df_original=df.copy()
df=df.reset_index()

print(df.shape)

(6247215, 41)


In [ ]:
#On se limite aux maisons et appartements

df = df[df['type_local'].isin(['Maison', 'Appartement'])]
print(df)

          id_mutation date_mutation  numero_disposition nature_mutation  \
17             2020-6    2020-07-02                   1           Vente   
31             2020-8    2020-07-01                   1           Vente   
32             2020-9    2020-07-02                   1           Vente   
36            2020-11    2020-07-03                   1           Vente   
37            2020-12    2020-07-02                   1           Vente   
...               ...           ...                 ...             ...   
20102723  2025-502875    2025-06-23                   1           Vente   
20102725  2025-502876    2025-06-25                   1           Vente   
20102726  2025-502877    2025-06-25                   1           Vente   
20102734  2025-502880    2025-06-27                   1           Vente   
20102735  2025-502881    2025-06-27                   1           Vente   

          valeur_fonciere  adresse_numero adresse_suffixe  \
17                77000.0           34

In [ ]:
#Nettoyage des données

#Rq : le code suivant permet de supprimer les lignes avec des valeurs manquantes de code commune ou d'id mais il n'y en a en fait pas
#df_clean = df.dropna(subset=['id_mutation', 'code_commune'])
#print(df_clean.shape)
#df_clean=df : aucune valeur manquante de commune ou d'id


#Prblm : certains biens s'étendent sur plusieurs communes (vidsiblement souvent des ensembles de champs ou de terrains non bâtis)
#Pour simplifier on supprime les observations concernées
#le tableau crée 

problematic_ids = df.groupby('id_mutation')['code_commune'].nunique()
problematic_ids = problematic_ids[problematic_ids > 1].index

print(problematic_ids.shape)
print('2020-2' in problematic_ids)
print('2020-7' in problematic_ids)

df_clean = df[~df['id_mutation'].isin(problematic_ids)]
print(df_clean.shape)


(2531,)
False
False
(6166434, 41)


In [ ]:
#les codes communes n'ont pas tous le même type ; certains sont des ints, d'autres des strings. On convertit l'ensemble en string pour plus de cohérence(on 
#ne peut pas faire l'opération inverse ; les codes corses commencent par 2A ou 2B !)

def inttostr(valeur):
    if isinstance(valeur, float):
        return str(valeur)
    if isinstance(valeur, int):
        return str(valeur)
    else:
        # Si c'est déjà une str, None, float, etc., on le retourne tel quel
        return valeur
print(inttostr('433'))

df_clean.loc[:,'code_commune'] = df_clean['code_commune'].apply(inttostr)
print(df_clean['code_commune'])

433
17           1130
31           1451
32           1364
36           1053
37           1177
            ...  
20102723    75111
20102725    75112
20102726    75116
20102734    75113
20102735    75105
Name: code_commune, Length: 6166434, dtype: object


In [ ]:
#Calcul du nombre d'annonces avec des identifiants distincts pour chaque commune

ventes_par_commune=df_clean.groupby("code_commune")["id_mutation"].nunique().reset_index()


#Résultats : on remarque on particulier que Toulouse est la commune comptant le plus de ventes entre 2020 et 2025(60398 au total, 43582 si on se restreint aux appartements et maisons) !!!

print(ventes_par_commune)

print(ventes_par_commune[ventes_par_commune['code_commune']=='2A004'])

print(ventes_par_commune['code_commune'])

print(ventes_par_commune['id_mutation'].max())

print(ventes_par_commune['id_mutation'].idxmax())

print(ventes_par_commune['id_mutation'][ventes_par_commune['id_mutation'].idxmax()])

print(ventes_par_commune['code_commune'][ventes_par_commune['id_mutation'].idxmax()])



      code_commune  id_mutation
0             1001           43
1             1002           17
2             1004         1107
3             1005          168
4             1006           14
...            ...          ...
33863        30352           52
33864        30353            2
33865        30354            3
33866        30355            8
33867        30356           67

[33868 rows x 2 columns]
      code_commune  id_mutation
33176        2A004         4318
0         1001
1         1002
2         1004
3         1005
4         1006
         ...  
33863    30352
33864    30353
33865    30354
33866    30355
33867    30356
Name: code_commune, Length: 33868, dtype: object
43582
11420
43582
31555


In [ ]:
#On récupère désormais les données de population ; cependant certaines valeurs ne sont pas renseignés. On remarque aussi que certaines communes ont des populations trop faibles
#pour que le ratio considéré ventes entre 2020 et 2050/population en 2019 ait du sens ; en particulier, la commune de Bezonvaux a zéro habitants !!! 
# On impose donc un nombre d'habitants minimum arbitraire, et on exclut les communes de populations trop faibles ou non renseignés

df_pop=pd.read_excel("popcommunes.xlsx")
print(df_pop)

cols = ["p19_pop"]
df_pop[cols].isna().sum()
lignes_na = df_pop[df_pop[cols].isna().any(axis=1)]
print(lignes_na)
print(lignes_na.shape)

       objectid  reg dep    cv codgeo                 libgeo  p13_pop  \
0        115658   52  85  8502  85062            Châteauneuf    968.0   
1        115659   26  58  5808  58300                   Urzy   1839.0   
2        115660   43  70  7012  70137  Chassey-lès-Montbozon    218.0   
3        115661   21  51  5123  51649      Vitry-le-François  13174.0   
4        115662   11  78  7811  78638         Vaux-sur-Seine   4749.0   
...         ...  ...  ..   ...    ...                    ...      ...   
34990    110251   31  62  6225  62327         Febvin-Palfart    574.0   
34991    110252   73  12  1201  12156            Montpeyroux    550.0   
34992    110253   54  86  8610  86138                Luchapt    275.0   
34993    110254   72  33  3320  33184                Générac    579.0   
34994    110255   72  24  2409  24354       La Roche-Chalais   2932.0   

       p14_pop  p15_pop  p16_pop  p17_pop  p18_pop  p19_pop  p20_pop  p21_pop  
0        993.0   1013.0   1027.0     1056  

In [ ]:
#On joint les données en se restreignant aux communes dont on connaît les populations et de populations pas trop faibles. Le min est arbitraire est peut être changé

df_pop.rename(columns={'codgeo': 'code_commune'}, inplace=True)

ventes_par_commune_par_habitant=pd.merge(ventes_par_commune, df_pop, on='code_commune')
ventes_par_commune_par_habitant = ventes_par_commune_par_habitant[
    (ventes_par_commune_par_habitant['p19_pop'].notna()) &
    (ventes_par_commune_par_habitant['p19_pop'] > 1000)
]
ventes_par_commune_par_habitant['ventes par habitants par commune']=ventes_par_commune_par_habitant['id_mutation']/ventes_par_commune_par_habitant['p19_pop']
print(ventes_par_commune_par_habitant['ventes par habitants par commune'])



print(ventes_par_commune_par_habitant['ventes par habitants par commune'].max())

print(ventes_par_commune_par_habitant['ventes par habitants par commune'].idxmax())
print(ventes_par_commune_par_habitant['code_commune'][ventes_par_commune_par_habitant['ventes par habitants par commune'].idxmax()])

1        0.087422
4        0.075633
15       0.051808
27       0.038562
30       0.058283
           ...   
30012    0.097814
30014    0.066615
30016    0.064635
30018    0.126162
30021    0.041164
Name: ventes par habitants par commune, Length: 8667, dtype: float64
0.9019908116385911
11022
38191


In [ ]:
#tentative de récupération des prix au m2
#Remarque : on ne considère que le ratio valeur foncière/surface batîe de la maison ou de l'appartement, puisqu'on ignore les dépendances ou autres terrains additionnels.
#on sur-estime donc la valeur au m2
df_clean = df_clean.copy()
print(df_clean.shape)

df_clean = df_clean[
    (df_clean['surface_reelle_bati'].notna()) &
    (df_clean['valeur_fonciere'].notna()) &
    (df_clean['surface_reelle_bati'] > 10)
]



df_clean['rapport valeur foncière et surface bâtie']=df_clean['valeur_fonciere']/df_clean['surface_reelle_bati']

print(df_clean['rapport valeur foncière et surface bâtie'])

print(df_clean['rapport valeur foncière et surface bâtie'].max())

print(df_clean['rapport valeur foncière et surface bâtie'].idxmax())

print(df_clean.loc[df_clean['rapport valeur foncière et surface bâtie'].idxmax()])

#problème ; valeurs monstrueuses de certains logements, visiblement des immeubles entiers plutôt que des logements(ci-dessus,immeuble en construction de presque 1 milliard...)


#ci-dessous moyennes et médianes. pas concluant vraisemblablement à cause de valeurs aberrantes du genre

moyenne_par_commune=df_clean.groupby('code_commune')['rapport valeur foncière et surface bâtie'].mean().reset_index()

print(moyenne_par_commune['rapport valeur foncière et surface bâtie'].max())

print(moyenne_par_commune['rapport valeur foncière et surface bâtie'].idxmax())

print(moyenne_par_commune['code_commune'].loc[moyenne_par_commune['rapport valeur foncière et surface bâtie'].idxmax()])



# Calcul de la médiane par commune
mediane_par_commune = df_clean.groupby('code_commune')['rapport valeur foncière et surface bâtie'].median().reset_index()

# Valeur maximale de la médiane
print(mediane_par_commune['rapport valeur foncière et surface bâtie'].max())

# Index de la commune avec la médiane maximale
print(mediane_par_commune['rapport valeur foncière et surface bâtie'].idxmax())

# Code de la commune avec la médiane maximale
print(mediane_par_commune['code_commune'][mediane_par_commune['rapport valeur foncière et surface bâtie'].idxmax()])


(6128046, 42)
0            669.565217
1           2286.885246
2           2220.625000
3           1966.406977
4           1666.666667
               ...     
6247210    11928.571429
6247211    11363.636364
6247212    14891.304348
6247213     9016.393443
6247214    11710.342553
Name: rapport valeur foncière et surface bâtie, Length: 6128046, dtype: float64
50800000.0
4802712
index                                                                 15145814
id_mutation                                                       2023-1321140
date_mutation                                                       2023-01-04
numero_disposition                                                           1
nature_mutation                             Vente en l'état futur d'achèvement
valeur_fonciere                                                    762000000.0
adresse_numero                                                            14.0
adresse_suffixe                                                       